In [104]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import regex as re
from splinter import Browser

In [105]:
url = "https://www.wayfair.com/appliances/sb0/built-in-dishwashers-c1874332.html"

In [106]:
# Set up Splinter
browser = Browser('chrome')

In [107]:
# 
browser.visit(url)

In [108]:
# Parse the HTML
html = browser.html
html_soup = BeautifulSoup(html, 'html.parser')

# Start

In [109]:

results = html_soup.find_all(attrs={"data-hb-id": "Grid.Item", "class":"kzv0b81_6101"})

In [139]:
for result in results:
    title = result.find('h2', class_='kb51y90_6101 kb51y91_6101 kb51y93_6101')
    
    if title:
        
        # Grab the maker
        box = result.find_all("p", attrs={"data-hb-id":"Text"})
        box_elements = []
        for x in box:
            box_elements.append(x.text)
        print(box_elements[5])
        print(box_elements[7])
    

    
            
        
            


    print('---')  # Separation between each dishwasher

16
5
---
15
6
---
16
4
---
16
5
---
14
8
---
15
6
---
14
4
---
16
5
---
16
6
---
16
6
---
16
4
---
14
8
---
14
60 decibel
---
12
2
---
15
6
---
14
5
---
14
4
---
15
3
---
14
3
---
16
5
---
12
3
---
12
4
---
15
6
---
14
6
---
14
4
---
16
5
---
14
6
---
Number of Place Settings
Number of Cycles
---
14
52 decibel
---
Number of Place Settings
Number of Cycles
---
16
5
---
14
6
---
14
51 decibel
---
14
7
---
14
4
---
16
5
---
8
6
---
8
3
---
16
5
---
12
3
---
12
6
---
8
3
---
16
5
---
15
7
---
14
4
---
16
5
---
16
5
---
16
5
---
---
---
---
---
---
---
---
---


In [151]:
dishwashers = []
for result in results:
    dw_info = {}
    # Extract the product rating
    title = result.find('h2', class_='kb51y90_6101 kb51y91_6101 kb51y93_6101')
    
    if title:
        
        # Grab the maker
        dw_info["maker"] = result.find(class_="oakhm610g_6101 oakhm614l_6101 nhya890_6101").text

        #Grab the size
        size = re.findall(r'[0-9]+"|[0-9]+ in', title.text)
        if size:
            dw_info["size"]=size[0][:2]
        else:
            dw_info["size"]=None

        # Grab noise level; looked it up and all dishwashers have a noiselevel between 30-100 dBA
        noise_level = re.findall(r"[0-9]+ dBA|[0-9]+ Decibel dBA", title.text)
        if noise_level:
            dw_info["noise_level"]=noise_level[0][:2]
            
        else:
            dw_info["noise_level"] = None

        # Grab color options
        color_info = result.find(class_="MoreOptionsText")
        if color_info:
            dw_info["num_colors"] = int(re.findall(r"\+[0-9]+",color_info.text)[0].replace("+","")) + 1
        else:
            dw_info["num_colors"] = 1

        # Grab the elements in the additional information box and store the elements in a list to check through 
        box = result.find_all("p", attrs={"data-hb-id":"Text"})
        box_elements = []
        for x in box:
            box_elements.append(x.text)

        # Grab the number of place settings
        if "Number of Place Settings" in box_elements:
            # Need to grab the index after that of the "Number of Place Settings"
            dw_info["num_place_settings"]= box_elements[box_elements.index("Number of Place Settings") +1]
        else:
            # If there isn't "Number of Place Settings" in the box, then put none
            dw_info["num_place_settings"] = None

        # Grab the number of cycles    
        if "Number of Cycles" in box_elements:
            # Need to grab the index after that of the "Number of Cycles"
            dw_info["num_cycles"]= box_elements[box_elements.index("Number of Cycles") +1]
        else:
            # If there isn't "Number of Cycles" in the box, then put none
            dw_info["num_cycles"] = None

        # Grab the price
        price_info = result.find_all(class_="oakhm627_6101 oakhm6y5_6101 oakhm610g_6101 oakhm6aj_6101")
        prices = []
        for x in price_info:
            y = x.text
            if re.search(r"\$", y):
                prices.append(y.replace("$",""))
        prices_int = [float(p.replace(",","")) for p in prices]
        if len(prices) ==2:
            dw_info["price"] = prices_int[0]
            dw_info["original_price"] = prices_int[1]
            dw_info["discount"] = prices_int[1]-prices_int[0]
        else:
            dw_info["price"] = prices_int[0]
            dw_info["original_price"] = None
            dw_info["discount"] = 0
        

        # Grab the rating
        rating_info = result.find(class_="_1xxktfu7_6101 _1xxktfu3_6101 _1xxktfu8_6101")
        if rating_info:
            rating_style = re.findall(r"width:[0-9]+|width: [0-9]+", rating_info["style"]) 
            dw_info["rating"]=int(rating_style[0][-2:])/20
        else: 
            dw_info["rating"] =None

        # Grab the number of ratings:
        num_ratings_info = result.find(class_="_1xxktfua_6101 undefined")
        if num_ratings_info:
            dw_info["num_ratings"] = num_ratings_info.text.replace("(", "").replace(")","")
        else:
            dw_info["num_ratings"]=None

        # Add the dishwasher to the list
        dishwashers.append(dw_info)

  

In [152]:
pd.DataFrame(dishwashers)

,maker,size,noise_level,num_colors,num_place_settings,num_cycles,price,original_price,discount,rating,num_ratings
0,GE Appliances,24,48,1,16,5,547.00,899.00,352.00,4.5,1446
1,Samsung,24,48,2,15,6,587.96,899.00,311.04,4.0,813
2,GE Appliances,24,52,4,16,4,448.04,679.00,230.96,4.5,1913
3,GE Appliances,24,50,4,16,5,497.00,779.00,282.00,4.5,1799
4,Frigidaire,24,49,2,14,8,629.00,1079.00,450.00,4.5,3744
5,LG,24,44,4,15,6,649.34,879.99,230.65,4.5,958
6,GE Appliances,24,55,4,14,4,404.98,599.00,194.02,4.0,806
7,GE Appliances,24,48,6,16,5,552.60,849.00,296.40,4.5,8161
8,ZLINE,18,None,12,16,6,999.95,NaN,0.00,4.5,210
9,ZLINE,24,None,8,16,6,1399.95,NaN,0.00,4.5,50


# Start parsing

In [102]:
browser.quit()